# Introduction

> Compare three different architectures models for classifying waste images from the TrashNet dataset.
- Fully Connected Networks (FC),
- Convolutional Neural Networks (CNN)
- Quantum Convolutional Neural Networks (QCNN)

> Then, evaluate the models classification:
- Accuracy
- Precision
- Recall
- F1-score
- Computational Efficiency
- Robustness

---

## 1. Import necessary libraries

In [2]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
from datasets import load_dataset

## 2. Load the TrashNet dataset

In [24]:
ds = load_dataset("garythung/trashnet")
print(ds)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 5054
    })
})


In [30]:
# Check the available splits in the dataset
print(ds.keys())


dict_keys(['train'])


## 3. Image and label preprocessing

In [32]:
def preprocess(example):
    image = tf.image.resize(example['image'], (64, 64))
    image = tf.image.convert_image_dtype(image, tf.float32)
    label = example['label']
    return image, label

# Convert Hugging Face Dataset to TensorFlow Dataset
def hf_to_tf_dataset(hf_dataset):
    def generator():
        for example in hf_dataset:
            yield example['image'], example['label']

    return tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(64, 64, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int64)
        )
    )

In [33]:
train_ds = hf_to_tf_dataset(ds['train']).map(lambda x, y: preprocess({'image': x, 'label': y})).shuffle(1000)

In [35]:
# Manually split 15% of the training data for validation
val_size = int(0.15 * len(ds['train']))
train_ds, valid_ds = train_ds.skip(val_size), train_ds.take(val_size)

In [36]:
# Batch and prefetch datasets
train_ds = train_ds.batch(32).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(32).prefetch(tf.data.AUTOTUNE)

## 4. Model Architectures

### 4.1 Fully Connected Neural Network Model

In [38]:
def build_fc_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(64, 64, 3)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

### 4.2 Convolutional Neural Network (CNN) Model

In [39]:
def build_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(6, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

### 4.3 Quantum Convolutional Neural Network (QCNN) Model

In [16]:
!pip install tensorflow==2.15.0 # Install a compatible version of tensorflow
!pip install cirq==1.3.0 # Install Cirq version 1.3.0 as per the issue description
!pip install -U tensorflow-quantum # Update tensorflow-quantum to the latest compatible version
!pip install sympy

In [17]:
# Import Libraries for Quantum
import tensorflow_quantum as tfq
import cirq
import sympy

In [37]:
# Quantum Convolution Layer with CZ and CX gates
def quantum_conv_layer(qubits, circuit, params):
    # RX Rotation
    for i in range(len(qubits)):
        circuit.append(cirq.rx(params[i])(qubits[i]))

    # CX gates
    for i in range(len(qubits) - 1):
        circuit.append(cirq.CX(qubits[i], qubits[i + 1]))

    # CZ gates
    for i in range(len(qubits) - 1):
        circuit.append(cirq.CZ(qubits[i], qubits[i + 1]))

    return circuit